In [26]:
#import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [27]:
tot_df = []
# for j,i in enumerate(np.arange(2019, 2021, 1)):
for j,i in enumerate(np.arange(2019, 2021, 1)):
    tot_df.append(pd.read_csv('../Data/Raw/CDC/CDCSTDcountysexagenew'+ 
                              str(i) +'.csv', sep = ',', skiprows = 4))
df = pd.concat(tot_df, axis = 0)
# df = pd.read_csv('../Data/Raw/CDC/CDCSTDcountysexrace2016.csv', skiprows = 4)

In [28]:
df.head()

,Indicator,Year,Geography,FIPS,Sex,Age Group,Cases,Rate per 100000
0,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,Unknown,Data suppressed,Data suppressed
1,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,65+,Data suppressed,Data suppressed
2,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,55-64,Data suppressed,Data suppressed
3,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,45-54,Data suppressed,Data suppressed
4,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,40-44,Data suppressed,Data suppressed


In [29]:
pd.unique(df['Year'])

array([2019, '2020 (COVID-19 Pandemic)'], dtype=object)

In [30]:
df.tail()

,Indicator,Year,Geography,FIPS,Sex,Age Group,Cases,Rate per 100000
283795,Chlamydia,2020 (COVID-19 Pandemic),"Ziebach County, SD",46137,Female,30-34,Data suppressed,Data suppressed
283796,Chlamydia,2020 (COVID-19 Pandemic),"Ziebach County, SD",46137,Female,25-29,Data suppressed,Data suppressed
283797,Chlamydia,2020 (COVID-19 Pandemic),"Ziebach County, SD",46137,Female,20-24,Data suppressed,Data suppressed
283798,Chlamydia,2020 (COVID-19 Pandemic),"Ziebach County, SD",46137,Female,15-19,Data suppressed,Data suppressed
283799,Chlamydia,2020 (COVID-19 Pandemic),"Ziebach County, SD",46137,Female,0-14,0,0.0


In [31]:
df = df.rename(columns = {'Cases':'cases',
                            'Rate per 100000':'ratePer100000',
                             'Indicator':'indicator',
                             'Year':'year',
                             'Geography':'geography',
                             'FIPS':'fips',
                             'Percent':'percent',
                             'Sex':'sex',
                             'Age Group' : 'ageGroup'})
df.head()

,indicator,year,geography,fips,sex,ageGroup,cases,ratePer100000
0,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,Unknown,Data suppressed,Data suppressed
1,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,65+,Data suppressed,Data suppressed
2,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,55-64,Data suppressed,Data suppressed
3,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,45-54,Data suppressed,Data suppressed
4,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,40-44,Data suppressed,Data suppressed


In [32]:
df.dtypes

indicator        object
year             object
geography        object
fips              int64
sex              object
ageGroup         object
cases            object
ratePer100000    object
dtype: object

In [33]:
df.loc[df["year"].str.startswith('2020',na=False), "year"] = "2020"

In [34]:
data_to_be_removed = ["cases", "ratePer100000"]

In [35]:
for col in data_to_be_removed:
    df.loc[df[col].str.contains("Data",na = False), col] = np.nan
    df[col] = df[col].str.replace(',','')

In [36]:
df.head()

,indicator,year,geography,fips,sex,ageGroup,cases,ratePer100000
0,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,Unknown,NaN,NaN
1,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,65+,NaN,NaN
2,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,55-64,NaN,NaN
3,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,45-54,NaN,NaN
4,Primary and Secondary Syphilis,2019,"Abbeville County, SC",45001,Male,40-44,NaN,NaN


In [37]:
df[["year","fips","cases","ratePer100000"]] = df[["year","fips","cases","ratePer100000"]].apply(pd.to_numeric)

In [38]:
df.dtypes

indicator         object
year               int64
geography         object
fips               int64
sex               object
ageGroup          object
cases            float64
ratePer100000    float64
dtype: object

In [39]:
df.drop(columns = 'geography', inplace = True)

In [40]:
df = df.pivot_table(index=['year','fips'], columns=['sex', 'indicator','ageGroup']).reset_index()
df.columns = df.columns.map('_'.join).str.strip('_')
df.head()

,year,fips,cases_Female_Chlamydia_0-14,cases_Female_Chlamydia_15-19,cases_Female_Chlamydia_20-24,cases_Female_Chlamydia_25-29,cases_Female_Chlamydia_30-34,cases_Female_Chlamydia_35-39,cases_Female_Chlamydia_40-44,cases_Female_Chlamydia_45-54,...,ratePer100000_Male_Primary and Secondary Syphilis_0-14,ratePer100000_Male_Primary and Secondary Syphilis_15-19,ratePer100000_Male_Primary and Secondary Syphilis_20-24,ratePer100000_Male_Primary and Secondary Syphilis_25-29,ratePer100000_Male_Primary and Secondary Syphilis_30-34,ratePer100000_Male_Primary and Secondary Syphilis_35-39,ratePer100000_Male_Primary and Secondary Syphilis_40-44,ratePer100000_Male_Primary and Secondary Syphilis_45-54,ratePer100000_Male_Primary and Secondary Syphilis_55-64,ratePer100000_Male_Primary and Secondary Syphilis_65+
0,2019,1001,1.0,83.0,81.0,30.0,20.0,3.0,3.0,2.0,...,0.0,53.9,61.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019,1003,3.0,185.0,211.0,92.0,28.0,15.0,3.0,0.0,...,0.0,0.0,17.9,32.0,0.0,15.3,0.0,0.0,0.0,0.0
2,2019,1005,4.0,65.0,49.0,23.0,9.0,5.0,1.0,1.0,...,0.0,135.3,112.5,175.3,0.0,109.8,0.0,0.0,0.0,0.0
3,2019,1007,0.0,39.0,31.0,13.0,3.0,2.0,1.0,0.0,...,0.0,0.0,129.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019,1009,1.0,41.0,86.0,19.0,14.0,3.0,1.0,2.0,...,0.0,0.0,0.0,0.0,59.7,0.0,0.0,0.0,0.0,0.0


In [41]:
df.tail()

,year,fips,cases_Female_Chlamydia_0-14,cases_Female_Chlamydia_15-19,cases_Female_Chlamydia_20-24,cases_Female_Chlamydia_25-29,cases_Female_Chlamydia_30-34,cases_Female_Chlamydia_35-39,cases_Female_Chlamydia_40-44,cases_Female_Chlamydia_45-54,...,ratePer100000_Male_Primary and Secondary Syphilis_0-14,ratePer100000_Male_Primary and Secondary Syphilis_15-19,ratePer100000_Male_Primary and Secondary Syphilis_20-24,ratePer100000_Male_Primary and Secondary Syphilis_25-29,ratePer100000_Male_Primary and Secondary Syphilis_30-34,ratePer100000_Male_Primary and Secondary Syphilis_35-39,ratePer100000_Male_Primary and Secondary Syphilis_40-44,ratePer100000_Male_Primary and Secondary Syphilis_45-54,ratePer100000_Male_Primary and Secondary Syphilis_55-64,ratePer100000_Male_Primary and Secondary Syphilis_65+
6182,2020,56037,1.0,26.0,26.0,15.0,6.0,3.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6183,2020,56039,0.0,11.0,12.0,10.0,5.0,2.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6184,2020,56041,0.0,8.0,5.0,7.0,3.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6185,2020,56043,0.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6186,2020,56045,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace('-','_')
df.head()

,year,fips,cases_female_chlamydia_0_14,cases_female_chlamydia_15_19,cases_female_chlamydia_20_24,cases_female_chlamydia_25_29,cases_female_chlamydia_30_34,cases_female_chlamydia_35_39,cases_female_chlamydia_40_44,cases_female_chlamydia_45_54,...,rateper100000_male_primary and secondary syphilis_0_14,rateper100000_male_primary and secondary syphilis_15_19,rateper100000_male_primary and secondary syphilis_20_24,rateper100000_male_primary and secondary syphilis_25_29,rateper100000_male_primary and secondary syphilis_30_34,rateper100000_male_primary and secondary syphilis_35_39,rateper100000_male_primary and secondary syphilis_40_44,rateper100000_male_primary and secondary syphilis_45_54,rateper100000_male_primary and secondary syphilis_55_64,rateper100000_male_primary and secondary syphilis_65+
0,2019,1001,1.0,83.0,81.0,30.0,20.0,3.0,3.0,2.0,...,0.0,53.9,61.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019,1003,3.0,185.0,211.0,92.0,28.0,15.0,3.0,0.0,...,0.0,0.0,17.9,32.0,0.0,15.3,0.0,0.0,0.0,0.0
2,2019,1005,4.0,65.0,49.0,23.0,9.0,5.0,1.0,1.0,...,0.0,135.3,112.5,175.3,0.0,109.8,0.0,0.0,0.0,0.0
3,2019,1007,0.0,39.0,31.0,13.0,3.0,2.0,1.0,0.0,...,0.0,0.0,129.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019,1009,1.0,41.0,86.0,19.0,14.0,3.0,1.0,2.0,...,0.0,0.0,0.0,0.0,59.7,0.0,0.0,0.0,0.0,0.0


In [43]:
df.head()

,year,fips,cases_female_chlamydia_0_14,cases_female_chlamydia_15_19,cases_female_chlamydia_20_24,cases_female_chlamydia_25_29,cases_female_chlamydia_30_34,cases_female_chlamydia_35_39,cases_female_chlamydia_40_44,cases_female_chlamydia_45_54,...,rateper100000_male_primary and secondary syphilis_0_14,rateper100000_male_primary and secondary syphilis_15_19,rateper100000_male_primary and secondary syphilis_20_24,rateper100000_male_primary and secondary syphilis_25_29,rateper100000_male_primary and secondary syphilis_30_34,rateper100000_male_primary and secondary syphilis_35_39,rateper100000_male_primary and secondary syphilis_40_44,rateper100000_male_primary and secondary syphilis_45_54,rateper100000_male_primary and secondary syphilis_55_64,rateper100000_male_primary and secondary syphilis_65+
0,2019,1001,1.0,83.0,81.0,30.0,20.0,3.0,3.0,2.0,...,0.0,53.9,61.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019,1003,3.0,185.0,211.0,92.0,28.0,15.0,3.0,0.0,...,0.0,0.0,17.9,32.0,0.0,15.3,0.0,0.0,0.0,0.0
2,2019,1005,4.0,65.0,49.0,23.0,9.0,5.0,1.0,1.0,...,0.0,135.3,112.5,175.3,0.0,109.8,0.0,0.0,0.0,0.0
3,2019,1007,0.0,39.0,31.0,13.0,3.0,2.0,1.0,0.0,...,0.0,0.0,129.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019,1009,1.0,41.0,86.0,19.0,14.0,3.0,1.0,2.0,...,0.0,0.0,0.0,0.0,59.7,0.0,0.0,0.0,0.0,0.0


In [47]:
df['fips'].value_counts()

1001     2
39071    2
39123    2
39125    2
39127    2
        ..
21175    2
21177    2
21179    2
56045    2
20071    1
Name: fips, Length: 3094, dtype: int64

In [45]:
df[df['fips']==30037]

,year,fips,cases_female_chlamydia_0_14,cases_female_chlamydia_15_19,cases_female_chlamydia_20_24,cases_female_chlamydia_25_29,cases_female_chlamydia_30_34,cases_female_chlamydia_35_39,cases_female_chlamydia_40_44,cases_female_chlamydia_45_54,...,rateper100000_male_primary and secondary syphilis_0_14,rateper100000_male_primary and secondary syphilis_15_19,rateper100000_male_primary and secondary syphilis_20_24,rateper100000_male_primary and secondary syphilis_25_29,rateper100000_male_primary and secondary syphilis_30_34,rateper100000_male_primary and secondary syphilis_35_39,rateper100000_male_primary and secondary syphilis_40_44,rateper100000_male_primary and secondary syphilis_45_54,rateper100000_male_primary and secondary syphilis_55_64,rateper100000_male_primary and secondary syphilis_65+
1570,2019,30037,0.0,1.0,2.0,0.0,0.5,0.5,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4663,2020,30037,0.0,1.5,2.0,1.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
df.to_csv("../Data/Clean/CDC/CDCSTDagesex20192020_clean.csv")